In [1]:
#!/usr/bin/python
# Import required libraries
import sys
import time
import RPi.GPIO as GPIO

In [2]:
def step(pins, direction=1, nsteps=1000, speed=1):
    # Use BCM GPIO references
    # instead of physical pin numbers
    GPIO.setmode(GPIO.BCM)
    StepPins = pins
    # Set all pins as output
    for pin in StepPins:
      GPIO.setup(pin,GPIO.OUT)
      GPIO.output(pin, False)

    # Define advanced sequence
    # as shown in manufacturers datasheet
    Seq = [[1,0,0,1],
           [1,0,0,0],
           [1,1,0,0],
           [0,1,0,0],
           [0,1,1,0],
           [0,0,1,0],
           [0,0,1,1],
           [0,0,0,1]]

    StepCount = len(Seq)
    StepDir = direction # Set to 1 or 2 for clockwise
                # Set to -1 or -2 for anti-clockwise

    # Read wait time from command line
    wait = 1

    if wait:
      WaitTime = float(wait)/float(1000)
    else:
      WaitTime = 10/float(1000)

    # Initialise variables
    StepCounter = 0

    # Start main loop
    for _ in range (nsteps):

      for pin in range(0, 4):
        xpin = StepPins[pin]
        if Seq[StepCounter][pin]!=0:
          GPIO.output(xpin, True)
        else:
          GPIO.output(xpin, False)

      StepCounter += StepDir

      # If we reach the end of the sequence
      # start again
      if (StepCounter>=StepCount):
        StepCounter = 0
      if (StepCounter<0):
        StepCounter = StepCount+StepDir

      # Wait before moving on
      time.sleep(WaitTime)

    GPIO.cleanup()
    

In [3]:
x = [17,22,23,24]
y = [6,13,19,26]
UP = 1
DOWN = -1
RIGHT = -1
LEFT= 1
nsteps = 2000

In [5]:
step(x, LEFT, 1000)

In [14]:
for i in range(20):
    step(x, RIGHT, nsteps)
    step(y, DOWN, nsteps)
    step(x, LEFT, nsteps)
    step(y, UP, nsteps)
    

KeyboardInterrupt: 

In [5]:
class Stage:
    def __init__(self, xpins, ypins):
        self.xpins = xpins
        self.ypins = ypins
        self.xpos = 0
        self.ypos = 0

    #TODO change to step to neg/pos int for direction
    def right(self, nsteps):
        self.step(self.xpins, direction=-1, nsteps=nsteps)
    def left(self, nsteps):
        self.step(self.xpins, direction=1, nsteps=nsteps)
    def up(self, nsteps):
        self.step(self.ypins, direction=1, nsteps=nsteps)
    def down(self, nsteps):
        self.step(self.ypins, direction=-1, nsteps=nsteps)
        
    def set_home(self):
        self.xpos = 0
        self.ypos = 0
    
    @property    
    def coordinates(self):
        return (self.xpos, self.ypos)
        
    def goto(self, x, y):
        xsteps = x - self.xpos
        # self.step(xsteps) 
        if xsteps < 0:
            self.right(abs(xsteps))
        elif xsteps > 0:
            self.left(xsteps)
        
        ysteps = y - self.ypos
        if ysteps < 0:
            self.down(abs(ysteps))
        elif ysteps > 0:
            self.up(ysteps)
        
    def home(self):
        # x
        # self.goto(0,0)
        if self.xpos < 0:
            self.left(abs(self.xpos))
        elif self.xpos > 0:
            self.right(self.xpos)
        # y
        if self.ypos < 0:
            self.up(abs(self.ypos))
        elif self.ypos > 0:
            self.down(self.ypos)
        
    def step(self, pins, direction=1, nsteps=1000, speed=1):
        # Use BCM GPIO references
        # instead of physical pin numbers
        GPIO.setmode(GPIO.BCM)
        StepPins = pins
        # Set all pins as output
        for pin in StepPins:
          GPIO.setup(pin,GPIO.OUT)
          GPIO.output(pin, False)

        # Define advanced sequence
        # as shown in manufacturers datasheet
        Seq = [[1,0,0,1],
               [1,0,0,0],
               [1,1,0,0],
               [0,1,0,0],
               [0,1,1,0],
               [0,0,1,0],
               [0,0,1,1],
               [0,0,0,1]]

        StepCount = len(Seq)
        StepDir = direction # Set to 1 or 2 for clockwise
                    # Set to -1 or -2 for anti-clockwise

        # Read wait time from command line
        wait = 1

        if wait:
          WaitTime = float(wait)/float(1000)
        else:
          WaitTime = 10/float(1000)

        # Initialise variables
        StepCounter = 0

        # Start main loop
        for _ in range (nsteps):

          for pin in range(0, 4):
            xpin = StepPins[pin]
            if Seq[StepCounter][pin]!=0:
              GPIO.output(xpin, True)
            else:
              GPIO.output(xpin, False)

          StepCounter += StepDir

          # If we reach the end of the sequence
          # start again
          if (StepCounter>=StepCount):
            StepCounter = 0
          if (StepCounter<0):
            StepCounter = StepCount+StepDir

          # Wait before moving on
          time.sleep(WaitTime)
        
        if pins == self.xpins:
            self.xpos += direction * nsteps
        else:
            self.ypos += direction * nsteps
        GPIO.cleanup()

        

In [6]:
stage = Stage([6,13,19,26], [17,22,23,24])

In [12]:
stage.up(1000)

In [16]:
stage.right(8000)

In [9]:
stage.home()

In [46]:
stage.coordinates

(0, 0)

In [51]:
stage.goto(20000, 0)

In [17]:
stage.coordinates

(-12000, 3000)

In [18]:
stage.set_home()

In [24]:
for _ in range(20):
    stage.goto(-2500, 2000)
    stage.goto(5000, -2500)
    stage.home()

KeyboardInterrupt: 

In [25]:
stage.home()

--2018-08-27 21:48:33--  https://gist.githubusercontent.com/python-friends/b51f4deb7d3e35ff516d1682460c3588/raw/GPIO.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.96.133
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.96.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 216 [text/plain]
Saving to: ‘RPi/GPIO.py’

RPi/GPIO.py         100%[=====================>]     216  --.-KB/s   in 0s     

Last-modified header missing -- time-stamps turned off.
2018-08-27 21:48:34 (1.21 MB/s) - ‘RPi/GPIO.py’ saved [216/216]



In [17]:
Ggp.cleanup()

clean
